In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, load_model
from keras.layers import ConvLSTM2D, BatchNormalization, Conv3D, Dense, Dropout, Flatten
from keras.utils.vis_utils import plot_model
import numpy as np
import tensorflow as tf

from constants import BATCH_SIZE, FACE_SHAPE, FRAMES_IN_SEQ, NUM_EPOCHS
from helpers import batch_gen, num_steps_per_epoch

Using TensorFlow backend.


In [2]:
train_gen = batch_gen("train", BATCH_SIZE)
train_steps = num_steps_per_epoch("train", BATCH_SIZE)
val_gen = batch_gen("val", BATCH_SIZE)
val_steps = num_steps_per_epoch("val", BATCH_SIZE)
test_gen = batch_gen("test", BATCH_SIZE)
test_steps = num_steps_per_epoch("test", BATCH_SIZE)

In [3]:
model = Sequential()
with tf.device("/gpu:0"):
    model.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                       input_shape=(FRAMES_IN_SEQ, 64, 64, 3),
                       padding='valid', return_sequences=True, activation="elu",
                       dropout=0.5))
    model.add(BatchNormalization())

with tf.device("/gpu:1"):
    model.add(Conv3D(filters=1, kernel_size=(5, 5, 5),
                   activation='elu',
                   padding='valid', data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(512, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation="elu"))
    model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()
plot_model(model, show_shapes=True, to_file="model.png")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 7, 62, 62, 32)     40448     
_________________________________________________________________
batch_normalization_1 (Batch (None, 7, 62, 62, 32)     128       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 3, 58, 58, 1)      4001      
_________________________________________________________________
flatten_1 (Flatten)          (None, 10092)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               5167616   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
__________

In [4]:
# callbacks
checkpoint = ModelCheckpoint(filepath="checkpoints/weights-{epoch:02d}-{val_acc:.2f}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=1, save_best_only=False)
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min", verbose=1)
callbacks = [checkpoint, early_stopping]

model.fit_generator(train_gen, steps_per_epoch=train_steps, epochs=NUM_EPOCHS, verbose=1,
                    callbacks=callbacks, validation_data=val_gen, validation_steps=val_steps)

Epoch 1/50
348/348 [==============================] - 92s 264ms/step - loss: 0.1192 - acc: 0.9702 - val_loss: 0.7370 - val_acc: 0.9484

Epoch 00001: saving model to checkpoints/weights-01-0.95-0.74.hdf5
Epoch 2/50
348/348 [==============================] - 74s 213ms/step - loss: 0.0978 - acc: 0.9910 - val_loss: 0.8220 - val_acc: 0.9484

Epoch 00002: saving model to checkpoints/weights-02-0.95-0.82.hdf5
Epoch 3/50
348/348 [==============================] - 74s 213ms/step - loss: 0.1728 - acc: 0.9882 - val_loss: 0.9251 - val_acc: 0.9406

Epoch 00003: saving model to checkpoints/weights-03-0.94-0.93.hdf5
Epoch 4/50
348/348 [==============================] - 74s 213ms/step - loss: 0.6973 - acc: 0.9557 - val_loss: 0.3487 - val_acc: 0.9781

Epoch 00004: saving model to checkpoints/weights-04-0.98-0.35.hdf5
Epoch 5/50
348/348 [==============================] - 74s 213ms/step - loss: 0.1394 - acc: 0.9913 - val_loss: 0.1495 - val_acc: 0.9906

Epoch 00005: saving model to checkpoints/weights-05-

In [6]:
model = load_model("checkpoints/weights-06-0.99-0.12.hdf5")
eval_results = model.evaluate_generator(test_gen, steps=test_steps, verbose=1)

18/18 [==============================] - 2s 105ms/step


In [8]:
print("Loss {}, accuracy {}".format(eval_results[0], eval_results[1]))

Loss 0.08303335812002564, accuracy 0.9947916666666666
